# Importing Libraries





In [ ]:
import math
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

path = "AILA_2019_dataset/relevance_judgments_statutes.txt"

f=open(path,'r')

# Making a Query Map

In [ ]:

temp_data1 = f.read()

temp_data = temp_data1.split("\n")

q_map={}

for line in temp_data:
    components=line.split(' ')
    if not components[0] in q_map:
        q_map[components[0]]={}
    q_map[components[0]][components[2]]={}
    q_map[components[0]][components[2]]=int(components[3])

In [ ]:
q_map

# Reading Results

In [ ]:
curr = pd.read_pickle(r'bert_tfidf_statutes.pickle')


# Understanding Relevance Data

In [ ]:
rel_map={}
for q_id in q_map.keys():
    rel_map[q_id]=0
    for co in q_map[q_id]:
        rel_map[q_id]+=int(q_map[q_id][co])

In [ ]:
rel_map

# Metric Calculation

In [ ]:
#For BM25
pat10=[]
meanp=[]
recip_rank=[]
for i in range(len(curr)):
    df=pd.DataFrame(curr[i])
    q_id='AILA_Q'+str(i+1)
    rel=0
    mAP=0
    num_rel=0
    r_rank=0
    for idx, row in df.iterrows():
        curr_id=row[0]
        rel+=q_map[q_id][curr_id]
        if idx==10:
            break
    for idx, row in df.iterrows():
        curr_id=row[0]
        if q_map[q_id][curr_id]==1:
            num_rel+=1
            if num_rel==1:
                r_rank=1/(idx+1)
            curr1=num_rel/(idx+1)
            mAP+=curr1
    meanp.append(mAP/num_rel)    
    pat10.append(rel/10)
    recip_rank.append(r_rank)

In [ ]:
#For BERT
pat10=[]
meanp=[]
recip_rank=[]
graph_x=[]
graph_y=[]
for q_id in curr.keys():
    df=pd.DataFrame(curr[q_id])
    rel=0
    mAP=0
    num_rel=0
    r_rank=0
    for idx, row in df.iterrows():
        curr_id=row[0]
        rel+=q_map[q_id][curr_id]
        if idx==9:
            break
    for idx, row in df.iterrows():
        curr_id=row[0]
        if q_map[q_id][curr_id]==1:
            num_rel+=1
            if num_rel==1:
                r_rank=1/(idx+1)
            curr1=num_rel/(idx+1)
            mAP+=curr1
    if num_rel==0:
        meanp.append(0)
    else:
        meanp.append(mAP/num_rel)    
    pat10.append(rel/10)
    recip_rank.append(r_rank)

In [ ]:
meanp_avg=0
recip_rank_avg=0
pat10_avg=0
for i in range (len(meanp)):
    meanp_avg+=meanp[i]
    recip_rank_avg+=recip_rank[i]
    pat10_avg+=pat10[i]
meanp_avg/=len(meanp)
recip_rank_avg/=len(recip_rank)
pat10_avg/=len(pat10)

In [ ]:
print(meanp_avg)
print(recip_rank_avg)
print(pat10_avg)

# Making Precision-Recall Plots

In [ ]:
graph_x=[0.0]
graph_y=[1.0]
q_id='AILA_Q13'
df=pd.DataFrame(curr[12])
num_rel=0

for idx, row in df.iterrows():
    curr_id=row[0]
    if q_map[q_id][curr_id]==1:
        num_rel+=1
    graph_y.append(num_rel/(idx+1))
    graph_x.append(num_rel/rel_map[q_id])
    if num_rel/rel_map[q_id]==1:
        break

In [ ]:
plt.plot(graph_x, graph_y)
 
# naming the x axis
plt.xlabel('Recall')
# naming the y axis
plt.ylabel('Precision')
 
# giving a title to my graph
plt.title('Precision Recall Plot')
 
# function to show the plot
plt.show()